In [1]:
import pandas as pd
import os

In [2]:
n4l_data_directory = "N4L_phenotypic_ontology_2016"

In [3]:
xlsx_config_path = "n4l-xlsx-parsing-config.tsv"

In [4]:
delimited_text_configs = [
    {
        "filename": "N4L_Taxonomy_20220802.tsv",
        "path": f"{n4l_data_directory}/N4L_Taxonomy_20220802.tsv",
        "id_column": "N4LID",
        "delimiter": "\t"
    },
    {
        "filename": "N4L_Taxonomy_20220802_pruned.tsv",
        "path": f"{n4l_data_directory}/N4L_Taxonomy_20220802_pruned.tsv",
        "id_column": "N4LID",
        "delimiter": "\t"
    },
    {
        "filename": "reference_id_mapping.csv",
        "path": f"{n4l_data_directory}/reference_id_mapping.csv",
        "id_column": "refid",
        "delimiter": ","
    }
]

In [5]:
def str_to_bool(val):
    """Convert common string values to boolean."""
    if pd.isna(val):
        return False
    return str(val).strip().lower() in {"1", "true", "yes", "y", "t"}

In [6]:
def process_transposed_sheet(df, id_column):
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    if id_column not in df.columns:
        raise ValueError(f"'{id_column}' not found in transposed headers")

    df = df.dropna(subset=[id_column])
    melted = df.melt(id_vars=[id_column], var_name="predicate", value_name="object_value")
    return melted

In [7]:
def process_standard_sheet(df, id_column):
    if id_column not in df.columns:
        raise ValueError(f"'{id_column}' not found in standard headers")

    df = df.dropna(subset=[id_column])
    melted = df.melt(id_vars=[id_column], var_name="predicate", value_name="object_value")
    return melted

In [8]:
melted_frames = []
melted_dropped_frames = []

In [9]:
for config in delimited_text_configs:
    df = pd.read_csv(config["path"], sep=config["delimiter"], low_memory=False)

    # Remove exact duplicates before anything else
    before = df.shape[0]
    df = df.drop_duplicates()
    after = df.shape[0]
    if after < before:
        print(f"[QC] {config['filename']} - Removed {before - after} fully duplicated rows")

    id_column = config["id_column"]
    if id_column not in df.columns:
        print(f"[ERROR] {config['filename']} - ID column '{id_column}' not found. Available columns: {df.columns.tolist()}")
        continue

    # Drop and log missing ID rows
    missing_id_rows = df[df[id_column].isna()]
    if not missing_id_rows.empty:
        print(f"[QC] {config['filename']} - Dropped {len(missing_id_rows)} rows missing '{id_column}'")
        melted_missing = missing_id_rows.melt(var_name="predicate", value_name="object_value")
        melted_missing["subject"] = None
        melted_missing["source_file"] = config["filename"]
        melted_missing["drop_reason"] = "missing_id"
        melted_dropped_frames.append(melted_missing)

    df = df.dropna(subset=[id_column])

    # Drop and log duplicated IDs
    duplicated_mask = df[id_column].duplicated(keep=False)
    if duplicated_mask.any():
        duplicated_ids = df.loc[duplicated_mask, id_column].unique()
        print(f"[DUPLICATES] {config['filename']} - {duplicated_mask.sum()} duplicate rows on '{id_column}' → {duplicated_ids.tolist()}")
        melted_dupes = df.loc[duplicated_mask].melt(var_name="predicate", value_name="object_value")
        melted_dupes["subject"] = df.loc[duplicated_mask, id_column].values.repeat(len(df.columns) - 1)
        melted_dupes["source_file"] = config["filename"]
        melted_dupes["drop_reason"] = "duplicate_id"
        melted_dropped_frames.append(melted_dupes)
        df = df[~duplicated_mask]
    else:
        print(f"[DUPLICATES] {config['filename']} - No duplicates in '{id_column}'")

    # Melt and append
    melted = df.melt(id_vars=[id_column], var_name="predicate", value_name="object_value")
    melted = melted.rename(columns={id_column: "subject"})
    melted["source_file"] = config["filename"]
    melted_frames.append(melted)
    print(f"[INFO] {config['filename']} → {melted.shape[0]} melted rows")


[DUPLICATES] N4L_Taxonomy_20220802.tsv - No duplicates in 'N4LID'
[INFO] N4L_Taxonomy_20220802.tsv → 1733490 melted rows
[DUPLICATES] N4L_Taxonomy_20220802_pruned.tsv - No duplicates in 'N4LID'
[INFO] N4L_Taxonomy_20220802_pruned.tsv → 1287953 melted rows
[DUPLICATES] reference_id_mapping.csv - No duplicates in 'refid'
[INFO] reference_id_mapping.csv → 117670 melted rows


In [10]:
# combined_delim_text_df = pd.concat(melted_frames, ignore_index=True)

In [11]:
# xlsx_sheet_configs = []

In [12]:
# for filename in xlsx_files:
#     print(f"Reading {filename}")
#     path = os.path.join(n4l_data_directory, filename)
#     try:
#         xls = pd.ExcelFile(path)
#         for sheet_name in xls.sheet_names:
#             xlsx_sheet_configs.append({
#                 "filename": filename,
#                 "sheet_name": sheet_name,
#                 "id_column": None,
#                 "skip": False
#             })
#     except Exception as e:
#         print(f"Error reading {filename}: {e}")

In [13]:
# xlsx_sheet_configs

In [14]:
# xlsx_sheet_configs_frame = pd.DataFrame(xlsx_sheet_configs)

In [15]:
# xlsx_sheet_configs_frame.to_csv("n4l-xlsx-parsing-config.tsv", sep="\t", index=False)

In [33]:
xlsx_sheet_configs = pd.read_csv(xlsx_config_path, sep="\t")

In [34]:
xlsx_sheet_configs

,filename,sheet_name,id_column,skip,requires_transposition,spo_already,note
0,complete.term.book_07.18.2013_CTP.xlsx,Sheet2,NaN,True,NaN,NaN,blank
1,complete.term.book_07.18.2013_CTP.xlsx,Sheet3,NaN,True,NaN,NaN,blank
2,protolog_normalization_categories_with_1000_DB...,Sheet1,NaN,True,NaN,NaN,Two-level class hierarchy of traits
3,protolog_normalization_categories_with_1000_KM...,Notes,NaN,True,NaN,NaN,NaN
4,N4L_Taxonomy_20220802_pruned.xlsx,N4L_Taxonomy_20220802_pruned,N4LID,False,False,NaN,NaN
5,N4L_Taxonomy_20220802.xlsx,N4L_Taxonomy_20220802_complete,N4LID,False,False,NaN,NaN
6,N4L_Taxonomy_20220802.xlsx,N4L_Taxonomy_20220802_pruned,N4LID,False,False,NaN,NaN
7,N4L_ID_to_NCBI_mappings.xlsx,N4L_NM.ID_to_EX.ID,Name N4LID,False,False,NaN,NaN
8,N4L_ID_to_NCBI_mappings.xlsx,N4L_NM.ID_to_NCBI_TaxID,Name N4LID,False,False,NaN,NaN
9,N4L_ID_to_NCBI_mappings.xlsx,N4L_REF.ID_to_DOCID,refid,False,False,NaN,NaN


In [35]:
melted_frames = []
melted_dropped_frames = []

In [36]:
for _, row in xlsx_sheet_configs.iterrows():
    print(row)

    skip = str_to_bool(row.get("skip", False))
    spo_already = str_to_bool(row.get("spo_already", False))
    requires_transposition = str_to_bool(row.get("requires_transposition", False))
    id_column = row.get("id_column")
    composite_columns = None

    if skip:
        continue

    file_path = os.path.join(n4l_data_directory, row["filename"])
    sheet_name = row["sheet_name"]

    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None if requires_transposition else 0)

        if requires_transposition:
            df = df.transpose()
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)

        if spo_already:
            if df.shape[1] != 3:
                print(
                    f"[ERROR] {row['filename']}:{sheet_name} - Expected 3 columns for SPO format, found {df.shape[1]}")
                continue
            df.columns = ["subject", "predicate", "object_value"]
            df = df.dropna(subset=["subject", "predicate", "object_value"])
            df["source_file"] = row["filename"]
            df["source_sheet"] = sheet_name
            melted_frames.append(df)
            print(f"[INFO] {row['filename']}:{sheet_name} (SPO) → {df.shape[0]} rows")
            continue

        if isinstance(id_column, str) and "|" in id_column:
            composite_columns = [col.strip() for col in id_column.split("|")]
            id_column = "_".join(composite_columns)

        if pd.isna(id_column) or id_column not in df.columns:
            print(
                f"[ERROR] {row['filename']}:{sheet_name} - ID column '{id_column}' not found. Available columns: {df.columns.tolist()}")
            continue

        df = df.drop_duplicates()

        if composite_columns:
            missing_cols = [col for col in composite_columns if col not in df.columns]
            if missing_cols:
                print(f"[ERROR] {row['filename']}:{sheet_name} - Missing composite ID columns: {missing_cols}")
                continue

            df = df.reset_index(drop=True)
            df[id_column] = df[composite_columns].astype(str).agg(
                lambda vals: "_".join([v for v in vals if v != "nan"]), axis=1
            )
            blank_ids = df[id_column] == ""
            if blank_ids.any():
                print(f"[QC] {row['filename']}:{sheet_name} - Dropped {blank_ids.sum()} rows with blank synthetic ID")
                df = df[~blank_ids]
            print(f"[INFO] Created synthetic ID column '{id_column}' from: {composite_columns}")

        df = df.dropna(subset=[id_column])

        full_dupes = df.duplicated()
        if full_dupes.any():
            print(f"[QC] {row['filename']}:{sheet_name} - Removed {full_dupes.sum()} fully duplicated rows")
            df = df[~full_dupes]

        duplicated_mask = df[id_column].duplicated(keep=False)
        duplicated_rows = df[duplicated_mask]

        if not duplicated_rows.empty:
            duplicated_ids = duplicated_rows[id_column].unique()
            print(
                f"[DUPLICATES] {row['filename']}:{sheet_name} - {len(duplicated_rows)} duplicate rows on '{id_column}' → {list(duplicated_ids)}")
            df = df[~duplicated_mask]
        else:
            print(f"[DUPLICATES] {row['filename']}:{sheet_name} - No duplicates in '{id_column}'")

        melted = df.melt(id_vars=[id_column], var_name="predicate", value_name="object_value")
        melted = melted.rename(columns={id_column: "subject"})
        melted = melted.dropna(subset=["subject", "predicate", "object_value"])
        melted["source_file"] = row["filename"]
        melted["source_sheet"] = sheet_name
        melted_frames.append(melted)
        print(f"[INFO] {row['filename']}:{sheet_name} → {melted.shape[0]} melted rows")

    except Exception as e:
        print(f"[ERROR] Failed processing {row['filename']}:{sheet_name} - {e}")


filename                  complete.term.book_07.18.2013_CTP.xlsx
sheet_name                                                Sheet2
id_column                                                    NaN
skip                                                        True
requires_transposition                                       NaN
spo_already                                                  NaN
note                                                       blank
Name: 0, dtype: object
filename                  complete.term.book_07.18.2013_CTP.xlsx
sheet_name                                                Sheet3
id_column                                                    NaN
skip                                                        True
requires_transposition                                       NaN
spo_already                                                  NaN
note                                                       blank
Name: 1, dtype: object
filename                  protolog_normaliza

/home/mark/gitrepos/metpo/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


[ERROR] protolog_normalization_categories_with_1000_DB.xlsx:Sheet2 - ID column 'rid_name.id' not found. Available columns: ['name', 'ex.id (or tx.id)', 'name.id', 'rid', 'Gram positive/negative', 'Gram stain/reaction', 'Gram type', 'Gram variable', 'acid-fastness', 'other stains', 'cell wall and membrane characteristics and capsules', 'cellular morphology (shape)', 'cellular morphology  (dimension 1)', 'cellular morphology  (dimension 2)', 'cellular morphology  (occurrence)', 'motility', 'flagella observations', 'division by', 'generation time', 'contain cellular inclusion bodies', 'do not contain contain cellular inclusion bodies', 'spore forming?', 'spore morphology (location)', 'spore morphology (shape)', 'spore morphology', 'pili, prosthecae, and other appendages', 'hyphal morphology', 'mycelium morphology', 'colony morphology', 'colony shape', 'colony color', 'colony diameter', 'colony margin', 'colony elevation', 'colony appearance, texture, and consistency', 'colony size', 'colo

/home/mark/gitrepos/metpo/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


[ERROR] protolog_normalization_categories_with_1000_DB.xlsx:Sheet3 - ID column 'rid_name.id' not found. Available columns: ['name', 'ex.id', 'name.id', 'rid', 'Gram positive/negative', 'Gram stain/reaction', 'Gram type', 'Gram variable', 'acid-fastness', 'other stains', 'cell wall and membrane characteristics and capsules', 'cellular morphology (shape)', 'cellular morphology  (dimension 1)', 'cellular morphology  (dimension 2)', 'cellular morphology  (dimension 3)', 'cellular morphology  (occurrence)', 'motility', 'flagella observations', 'division by', 'generation/doubling time', 'contain cellular inclusion bodies', 'do not contain contain cellular inclusion bodies', 'spore forming?', 'spore morphology (location)', 'spore morphology (shape)', 'spore morphology', 'pili, prosthecae, and other appendages', 'hyphal morphology', 'mycelium morphology', 'thallus morphology', 'colony morphology', 'colony shape', 'colony color', 'colony diameter', 'colony margin', 'colony elevation', 'colony a

/home/mark/gitrepos/metpo/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


[ERROR] protolog_normalization_categories_with_1000_KMP.xlsx:EffectRIDProtos(rid.2300 up) - ID column 'rid_name.id' not found. Available columns: ['name', 'ex.id (or tx.id)', 'name.id', 'rid', 'Gram positive/negative', 'Gram stain/reaction', 'Gram type', 'Gram variable', 'acid-fastness', 'other stains', 'cell wall and membrane characteristics and capsules', 'cellular morphology (shape)', 'cellular morphology  (dimension 1)', 'cellular morphology  (dimension 2)', 'cellular morphology  (occurrence)', 'motility', 'flagella observations', 'division by', 'generation time', 'contain cellular inclusion bodies', 'do not contain contain cellular inclusion bodies', 'spore forming?', 'spore morphology (location)', 'spore morphology (shape)', 'spore morphology', 'pili, prosthecae, and other appendages', 'hyphal morphology', 'mycelium morphology', 'colony morphology', 'colony shape', 'colony color', 'colony diameter', 'colony margin', 'colony elevation', 'colony appearance, texture, and consistency

/home/mark/gitrepos/metpo/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


[ERROR] protolog_normalization_categories_with_1000_KMP.xlsx:Sheet2 - ID column 'rid_name.id' not found. Available columns: ['name', 'ex.id', 'name.id', 'rid', 'Gram positive/negative', 'Gram stain/reaction', 'Gram type', 'Gram variable', 'acid-fastness', 'other stains', 'cell wall and membrane characteristics and capsules', 'cellular morphology (shape)', 'cellular morphology  (dimension 1)', 'cellular morphology  (dimension 2)', 'cellular morphology  (occurrence)', 'motility', 'flagella observations', 'division by', 'generation/doubling time', 'contain cellular inclusion bodies', 'do not contain contain cellular inclusion bodies', 'spore forming?', 'spore morphology (location)', 'spore morphology (shape)', 'spore morphology', 'pili, prosthecae, and other appendages', 'hyphal morphology', 'mycelium morphology', 'colony morphology', 'colony shape', 'colony color', 'colony diameter', 'colony margin', 'colony elevation', 'colony appearance, texture, and consistency', 'colony size', 'colon

In [22]:
# Combine all into one frame
combined_df = pd.concat(melted_frames, ignore_index=True)

In [23]:
combined_df.shape


(3736012, 5)

In [24]:
combined_df = combined_df.drop_duplicates()

In [25]:
combined_df.shape

(3736012, 5)

In [26]:
combined_df = combined_df.dropna(subset=["subject", "predicate", "object_value"])


In [27]:
combined_df.shape

(3736012, 5)

In [28]:
combined_df

,subject,predicate,object_value,source_file,source_sheet
0,rid.1093_nm.3736,refid,rid.1093,article_download_status_20161222.xlsx,all_protologs
1,rid.1093_nm.3737,refid,rid.1093,article_download_status_20161222.xlsx,all_protologs
2,rid.5447_nm.3315,refid,rid.5447,article_download_status_20161222.xlsx,all_protologs
3,rid.5447_nm.11022,refid,rid.5447,article_download_status_20161222.xlsx,all_protologs
4,rid.5447_nm.3314,refid,rid.5447,article_download_status_20161222.xlsx,all_protologs
...,...,...,...,...,...
3736007,nm.8017,note,The descriptions of Murinilabilia sulmonicolor...,protolog_normalization_categories_with_1000_KM...,Sheet2
3736008,nm.2199,note,The description of Acidithiobacillus thiooxida...,protolog_normalization_categories_with_1000_KM...,Sheet2
3736009,nm.5724,note,In addition to the description of the genus,protolog_normalization_categories_with_1000_KM...,Sheet2
3736010,rid.4142_nm.1005,note,no true protolog in this reference,protolog_normalization_categories_with_1000_KM...,Sheet2


In [ ]:
combined_dropped_df = pd.concat(melted_dropped_frames, ignore_index=True)

In [ ]:
combined_dropped_df.shape

In [ ]:
combined_dropped_df = combined_dropped_df.drop_duplicates()

In [ ]:
combined_dropped_df.shape

In [ ]:
combined_dropped_df